In [1]:
import pandas as pd

# --- 1. 加载数据集 ---
print("--- 1. 加载数据集 ---")
try:
    documents_df = pd.read_csv('/Users/aloez/Downloads/documents.csv')
    annotations_df = pd.read_csv('/Users/aloez/Downloads/annotations.csv')
    print("数据集加载成功！")
    print(f"documents.csv 包含 {len(documents_df)} 条记录。")
    print(f"annotations.csv 包含 {len(annotations_df)} 条记录。")

except FileNotFoundError:
    print("错误：请确保 'documents.csv' 和 'annotations.csv' 文件在当前目录下或提供正确路径。")
    exit() # 如果文件未找到，程序退出

# --- 2. 初步查看数据结构 ---
print("\n--- 2. 初步查看 documents.csv 的前几行 ---")
print(documents_df.head())
print("\n--- 2. 初步查看 annotations.csv 的前几行 ---")
print(annotations_df.head())

# 检查列名，确认是否存在 'native' 和 'tag'
print("\n--- documents.csv 的列名 ---")
print(documents_df.columns.tolist())
print("\n--- annotations.csv 的列名 ---")
print(annotations_df.columns.tolist())

# 请根据实际输出，确认 `document_id` 或 `text_id` 等用于关联两个表的公共ID字段
# 通常会有一个唯一的ID来连接文档信息和其对应的错误标注
# 假设是 'document_id'，如果不是，请根据实际列名修改
document_id_col_doc = 'document_id' # 假设 documents.csv 中的文档ID列名
document_id_col_anno = 'document_id' # 假设 annotations.csv 中的文档ID列名

if document_id_col_doc not in documents_df.columns:
    print(f"\n警告：在 documents.csv 中未找到列 '{document_id_col_doc}'。请检查实际列名。")
if document_id_col_anno not in annotations_df.columns:
    print(f"警告：在 annotations.csv 中未找到列 '{document_id_col_anno}'。请检查实际列名。")


# --- 3. 筛选目标学习者数据 ---
print("\n--- 3. 筛选目标学习者数据 ---")

# 筛选母语为汉语 ('zho') 和英语 ('eng') 的学习者
# 论文中通常使用ISO 639-2/3 代码，'zho'代表Chinese，'eng'代表English。
# 请注意：如果您的数据集中使用了不同的编码（例如 'Chinese', 'English'），请修改这里的列表。
target_languages = ['zho', 'eng']
target_learners_df = documents_df[documents_df['native'].isin(target_languages)]

print(f"筛选出 {len(target_learners_df)} 条来自目标母语学习者的记录。")
print("目标学习者的母语分布：")
print(target_learners_df['native'].value_counts())

# 提取目标学习者的 document_id (或其他唯一标识符)
target_document_ids = target_learners_df[document_id_col_doc].unique()
print(f"提取出 {len(target_document_ids)} 个目标学习者的唯一文档ID。")


# --- 4. 在 annotations 文件中定位动词体（Asp）错误 ---
print("\n--- 4. 在 annotations 文件中定位动词体（Asp）错误 ---")

# 查找所有 tag 字段为 'Asp' 的错误记录
# 根据论文，'Asp' 是动词体的标签。
# 如果初步检查中未出现，请务必在运行此代码后，手动检查 annotations.csv 的完整内容，
# 或尝试搜索其他可能的动词体相关标签（如'Aspect', 'VerbAspect'等）。
aspect_errors_df = annotations_df[annotations_df['tag'] == 'Asp'].copy() # 使用.copy()避免SettingWithCopyWarning

print(f"在 annotations.csv 中找到 {len(aspect_errors_df)} 条 'Asp' 错误记录。")
if len(aspect_errors_df) == 0:
    print("警告：未找到任何 'Asp' 标签的错误。请再次确认 'annotations.csv' 中的 'tag' 列是否存在此标签，或是否存在其他表示动词体的标签。")
    print("annotations.csv 中 tag 列出现的前10个标签类型：")
    print(annotations_df['tag'].value_counts().head(10)) # 打印前10个最常见的tag，帮助您识别

# --- 5. 关联目标学习者数据与动词体错误标注 ---
print("\n--- 5. 关联目标学习者数据与动词体错误标注 ---")

# 将 aspect_errors_df 进一步筛选，只保留属于目标学习者的错误
target_aspect_errors_df = aspect_errors_df[
    aspect_errors_df[document_id_col_anno].isin(target_document_ids)
].copy() # 使用.copy()避免SettingWithCopyWarning

print(f"最终筛选出 {len(target_aspect_errors_df)} 条来自目标学习者的动词体（Asp）错误。")

if len(target_aspect_errors_df) > 0:
    print("\n来自目标学习者的动词体错误示例（前5条）：")
    print(target_aspect_errors_df.head())

    # （可选）合并 learning_native 信息到错误记录中，方便后续分析
    # 确保合并时使用的 document_id 是共通的
    merged_errors_df = pd.merge(
        target_aspect_errors_df,
        target_learners_df[[document_id_col_doc, 'native']],
        left_on=document_id_col_anno,
        right_on=document_id_col_doc,
        how='left'
    )
    # 删除重复的 document_id 列，如果它们名称不同的话
    if document_id_col_doc != document_id_col_anno:
        merged_errors_df = merged_errors_df.drop(columns=[document_id_col_doc])

    print("\n合并学习者母语信息后的动词体错误示例（前5条）：")
    print(merged_errors_df.head())

    # 查看这些错误的母语分布
    print("\n目标学习者动词体错误的母语分布：")
    print(merged_errors_df['native'].value_counts())

    # 现在 merged_errors_df 包含了您需要的所有信息，可以保存以便后续分析
    merged_errors_df.to_csv('target_aspect_errors.csv', index=False, encoding='utf-8-sig')
    print("\n已将筛选出的目标动词体错误保存到 'target_aspect_errors.csv'。")

else:
    print("\n没有找到来自目标学习者的动词体（Asp）错误。请检查标签名称是否正确或数据集内容。")

--- 1. 加载数据集 ---
数据集加载成功！
documents.csv 包含 2004 条记录。
annotations.csv 包含 41410 条记录。

--- 2. 初步查看 documents.csv 的前几行 ---
   id subcorpus native language_background level  words  sentences
0   1     RULEC    eng                  HL    AM    431         22
1   3     RULEC    eng                  HL    AM    245         17
2   5     RULEC    eng                  FL    AM    472         22
3   6     RULEC    eng                  FL    IH    319         24
4   7     RULEC    eng                  HL    AL     44          2

--- 2. 初步查看 annotations.csv 的前几行 ---
       id  sentence_id              tag               quote  \
0    7944         1002            ortho           окружющей   
1  101216         1006          agrcase        промышленным   
2   18818         1009           syntax         воздействую   
3  119778         1009     ortho,altern  желудожно-кишечный   
4   18819         1012  syntax,transfer                 это   

           correction  start  end annotation_source  
0       

KeyError: 'document_id'

In [2]:
import pandas as pd

# --- 1. 定义文件路径和关键列名 ---
# 请根据您实际的文件名和列名进行修改
all_asp_errors_file = '/Users/aloez/Downloads/asp（动词体）.csv'  # 假设您将所有Asp错误保存到这个文件
documents_file = '/Users/aloez/Downloads/native.csv'          # 包含学习者母语信息的documents文件

# 这两个文件都有一个名为 'id' 的列用于关联
common_id_column_name = 'id'
native_language_column_name = 'native' # documents.csv中指示母语的列名
tag_column_name = 'tag' # Asp错误文件中的错误类型列名

# --- 2. 加载数据 ---
print("--- 2. 加载数据 ---")
try:
    # 加载包含所有Asp错误的CSV
    all_asp_errors_df = pd.read_csv(all_asp_errors_file)
    print(f"成功加载 '{all_asp_errors_file}'，包含 {len(all_asp_errors_df)} 条Asp错误记录。")

    # 加载包含学习者元数据的documents.csv
    documents_df = pd.read_csv(documents_file)
    print(f"成功加载 '{documents_file}'，包含 {len(documents_df)} 条文档记录。")

except FileNotFoundError as e:
    print(f"错误：文件未找到。请检查文件名和路径是否正确：{e}")
    exit() # 如果文件未找到，程序退出

# --- 3. 检查关键列是否存在 ---
if common_id_column_name not in all_asp_errors_df.columns:
    print(f"警告：'{all_asp_errors_file}' 中未找到关联列 '{common_id_column_name}'。当前列有: {all_asp_errors_df.columns.tolist()}")
    print("请根据实际列名修改 'common_id_column_name' 变量。")
    exit()

if common_id_column_name not in documents_df.columns:
    print(f"警告：'{documents_file}' 中未找到关联列 '{common_id_column_name}'。当前列有: {documents_df.columns.tolist()}")
    print("请根据实际列名修改 'common_id_column_name' 变量。")
    exit()

if native_language_column_name not in documents_df.columns:
    print(f"警告：'{documents_file}' 中未找到母语列 '{native_language_column_name}'。当前列有: {documents_df.columns.tolist()}")
    print("请根据实际列名修改 'native_language_column_name' 变量。")
    exit()

# --- 4. 合并数据框 ---
print("\n--- 4. 合并 Asp 错误与学习者母语信息 ---")
# 只选择 documents_df 中我们需要的列 (ID 和 native) 来合并，避免不必要的列
learner_native_info = documents_df[[common_id_column_name, native_language_column_name]].copy()

# 执行合并操作
# left_on 和 right_on 指定了两个数据框中用于合并的列
merged_asp_errors_df = pd.merge(
    all_asp_errors_df,
    learner_native_info,
    left_on=common_id_column_name,
    right_on=common_id_column_name,
    how='left' # 'left' 合并保留所有 Asp 错误，并添加匹配的母语信息
)

print(f"合并后的数据框包含 {len(merged_asp_errors_df)} 条记录。")
print("合并后的数据框前几行示例（包含 native 列）：")
print(merged_asp_errors_df.head())

# 检查合并后是否有缺失的 native 信息（可能意味着某些 document_id 在 documents.csv 中找不到）
missing_native_count = merged_asp_errors_df[native_language_column_name].isnull().sum()
if missing_native_count > 0:
    print(f"警告：有 {missing_native_count} 条 Asp 错误记录在 documents.csv 中未能找到对应的母语信息。")

# --- 5. 按母语筛选并分离数据 ---
print("\n--- 5. 按母语筛选并分离数据 ---")

# 确认母语编码，如 'chi' 和 'eng'
# 您可以先查看 merged_asp_errors_df['native'].value_counts() 来确认实际的母语编码
print("合并数据中母语分布：")
print(merged_asp_errors_df[native_language_column_name].value_counts())

chinese_asp_errors_df = merged_asp_errors_df[merged_asp_errors_df[native_language_column_name] == 'chi'].copy()
english_asp_errors_df = merged_asp_errors_df[merged_asp_errors_df[native_language_column_name] == 'eng'].copy()

print(f"筛选出 {len(chinese_asp_errors_df)} 条来自汉语学习者的 Asp 错误记录。")
print(f"筛选出 {len(english_asp_errors_df)} 条来自英语学习者的 Asp 错误记录。")

# --- 6. 保存分离后的数据（可选，但推荐） ---
print("\n--- 6. 保存分离后的数据 ---")
chinese_asp_errors_df.to_csv('chinese_asp_errors_with_native.csv', index=False, encoding='utf-8-sig')
english_asp_errors_df.to_csv('english_asp_errors_with_native.csv', index=False, encoding='utf-8-sig')
print("已将汉语学习者的 Asp 错误保存到 'chinese_asp_errors_with_native.csv'。")
print("已将英语学习者的 Asp 错误保存到 'english_asp_errors_with_native.csv'。")

print("\n数据已准备就绪，可以进行下一步的精细分类和定性分析了！")

--- 2. 加载数据 ---
成功加载 '/Users/aloez/Downloads/asp（动词体）.csv'，包含 1487 条Asp错误记录。
成功加载 '/Users/aloez/Downloads/native.csv'，包含 1064 条文档记录。

--- 4. 合并 Asp 错误与学习者母语信息 ---
合并后的数据框包含 1487 条记录。
合并后的数据框前几行示例（包含 native 列）：
       id  sentence_id             tag                   quote  \
0    9933         7002       asp,morph                   Стоив   
1    7468        13001      constr,asp              имеет силь   
2    8329        14003             asp      будет рассказывать   
3  124608        15001             asp                 положит   
4    8897        23016  constr,lex,asp  приехали в такие места   

                correction  start  end annotation_source native  
0                    Строя      0    1            manual    NaN  
1                 начинает      5    7            manual    NaN  
2                расскажет      2    4            manual    NaN  
3                   кладет      3    4            manual    NaN  
4  побывали в таких местах     12   16            manual    NaN